#### 一. 邻接矩阵
1. 什么是邻接矩阵  
  1. 邻接矩阵是图的一种表示方法, 能表示出顶点和边的关系  
  2. 邻接矩阵比邻接表在时间性能上更好, 因为大量查找变得操作可以通过矩阵直接定位, 而邻接表只能遍历顶点所在链表
1. 邻接矩阵 : 使用二维数组表示图中顶点和边的关系  
 1. 无向图 : $A[i][j]$=1表示两个定点之间存在边. 无向图的邻接矩阵是对称阵  
 2. 有向图 : $A[i][j]$=1表示两个定点之间存在从定点$i$ 到顶点$j$ 的边  
 3. 网络 : $A[i][j]$=n表示顶点之间边的权值
 
2. 邻接表编程实现的功能    
  1. 邻接表使用二维矩阵表示边集,一维向量表示顶点集
  3. 顶点上的操作 :  
    1. 获取顶点的一些信息  
    2. 插入新顶点 :  
     插入新顶点时, 顶点集增加, 边集中的入边和出边要设置成None  
    3. 删除顶点 :  
     删除顶点一切相关的边, 并修改相关顶点的入度出度
  4. 边的操作  
    1. 插入新边  
    2. 删除边  
     插入删除边都要修改对应的出度入度

In [1]:
###########################
###### 顶点和边的状态 #######
###########################
from enum import Enum
class Vstatus(Enum):
    '''顶点状态'''
    UNDISCOVERED = 1
    DISCOVERED = 2
    VISITED = 3
class EType(Enum):
    '''边的状态'''
    UNDETERMINED = 1
    TREE = 2
    CROSS = 3
    FORWARD = 4
    BACKWARD = 5

In [2]:
import sys

class Vertex(object):
    '''顶点'''
    def __init__(self,data,in_degree = 0,out_degree = 0,status = Vstatus.UNDISCOVERED,d_time = -1,
                f_time = -1,parent = -1,priority = sys.maxint):
        self.data = data
        self.in_degree = in_degree  # 入度
        self.out_degree = out_degree # 出度
        self.status = status
        self.d_time = d_time  # discover time
        self.f_time = f_time  # finish time
        self.parent = parent  # 顶点在遍历树中的父节点
        self.priority = priority # 顶点在遍历树中的优先级
        
class Edge(object):
    '''边'''
    def __init__(self,data,weight=-1,e_type=EType.UNDETERMINED):
        self.data = data
        self.weight = weight
        self.e_type = e_type
    
class Graph_Matrix(object):
    '''邻接矩阵表示的图'''
    def __init__(self):
        self.n = 0 # 顶点数
        self.e = 0 # 边数
        self.V = [] # 顶点集 : 一维数组
        self.E = [] # 边集 : 二维数组       
        
    ##### 顶点操作 
    def insert_vertex(self,vertex_data):
        '''插入顶点, 将其入边, 出边设成None'''
        self.n = self.n + 1
        # 出边设成None
        self.E.append([None for i in range(self.n)])
        # 入边设成None
        for i in range(self.n): 
            self.E[i].append(None)
        self.V.append(Vertex(vertex_data))
        
    def remove_vertes(self,i):
        '''删除顶点时,既要从顶点集中删除, 又要删除其关联变, 对应入度出度减1'''
        for j in range(self.n):
            self.E[j] = self.E[j][:i] + self.E[j][i+1:]  # 使用列表切片删除列表index下的元素
            self.V[j].in_degree = self.V[j].out_degree-1 # 出度减1
        for j in range(self.n):
            if self.exist_edge(i,j):
                self.V[j].in_degree = self.V[j].in_degree-1 # 入度减1
        self.E = self.E[:i]+self.E[i+1:]  # 删除该顶点所有出边集合
        v_remove = self.V[i]
        self.V = self.V[:i]+self.V[i+1:]  
        return v_remove
    
    ### 边操作
    def exist_edge(self,i,j):
        return self.E[i][j] is not None
    def insert_edge(self,edge_data,w,i,j,bidirect_edge=False):
        '''无向图, 增加变需要设置bidirect_edge=True'''
        if self.exist_edge(i,j):  # 确保边尚不存在
            print 'return'
            return 
        if not bidirect_edge:
            self.E[i][j] = Edge(edge_data,w)
            self.V[i].out_degree = self.V[i].out_degree+1
            self.V[j].in_degree = self.V[j].in_degree+1
        else:
            self.E[i][j] = Edge(edge_data,w)
            self.E[j][i] = Edge(edge_data,w)
            self.V[i].out_degree = self.V[i].out_degree+1
            self.V[i].in_degree = self.V[i].in_degree+1
            self.V[j].in_degree = self.V[j].in_degree+1
            self.V[j].out_degree = self.V[j].out_degree+1
        self.e = self.e+1 
        
    def remove_edge(self,i,j):
        '''删除边'''
        e_bak = self.E[i][j]
        self.E[i][j] = None
        e = e-1
        self.V[i].out_degree = self.V[i].out_degree-1
        self.V[j].in_degree = self.V[j].in_degree-1
        return e_bak

#### 二. 遍历
1. 深度优先    
  1. 深度优先遍历, 优先选取最后一个被访问到的顶点的邻居  
  2. 以顶点$s$为基点的$DFS$搜索, 首先访问$s$, 再从所有尚未访问到的邻居中任取其一, 并以之为基点, 再次递归进行$DFS$搜索  
   因此, 顶点被访问到的次序, 类似于树的先序遍历; 而各顶点被访问完毕的次序, 类似后序遍历  
  3. 如下程序展示了深度优先遍历如下图  
  <img src='img/shenduyouxian.png' height='30%' width='30%'>
2. 广度优先  
  1. 广度优先遍历, 越早被访问到的顶点, 其邻居优先被访问  
  2. 广度优先遍历在发现顶点后立刻访问, 所以顶点的d_time(发现时间)和f_time(完成时间)相等.  
   而深度优先遍历不同, 深度优先在发现顶点后, 不能立即访问, 先把自己和其邻居入栈, 等待最后回溯到该顶点时, 才能弹栈访问

In [3]:
class GraphTraverse(object):
    @staticmethod
    def DFS(start,graph,visit):
        '''start: 从哪个顶点开始遍历, 因为存在多连通分量的情况, 所以需要遍历整个节点看师傅都被访问到了
           graph: 需要被遍历的图
           visit: 访问函数
                `visit(current_idx,current_vertex)`: 2个参数, 顶点index和顶点对象
        '''
        edge_matrix = graph.E
        vertexs = graph.V
        n = graph.n  # 顶点个数
        
        clock = 1
        stack = []
        
        traverse_idx_order = range(start,n)+range(0,start) # 从开始位置向后遍历
        
        for start in traverse_idx_order:
            if vertexs[start].status == Vstatus.UNDISCOVERED:
                stack.append(start)
                vertexs[start].d_time = clock
                clock = clock + 1
            while len(stack) >0:
                s = stack[-1]
                # 所有邻居节点
                neighbors_idxs = [i for i in range(n) if edge_matrix[s][i] is not None]
                undiscover_neighbors = [i for i in neighbors_idxs if vertexs[i].status==Vstatus.UNDISCOVERED]
                disovered_neighbors = [i for i in neighbors_idxs if vertexs[i].status==Vstatus.DISCOVERED]
                visited_neighbors = [i for i in neighbors_idxs if vertexs[i].status==Vstatus.VISITED]

                # 1.如果栈顶顶点没有'未访问'的邻居, 则可弹栈进行访问
                if len(undiscover_neighbors)==0:
                    current_idx = stack.pop() # 弹栈
                    current_vertex = vertexs[current_idx]
                    current_vertex.status = Vstatus.VISITED
                    current_vertex.f_time = clock
                    visit(current_idx,current_vertex)
                # 2.未发现的邻居节点
                else:
                    for i in undiscover_neighbors:
                        stack.append(i) # 未发现的邻居节点先入栈,但不访问.
                        vertexs[i].status = Vstatus.DISCOVERED
                        vertexs[i].parent = s
                        vertexs[i].d_time = clock
                        edge_matrix[s][i].status = EType.TREE

                # 3.已发现的邻居节点
                for i in disovered_neighbors:
                    edge_matrix[s][i].status = EType.BACKWARD

                # 4.已访问的邻居节点
                for i in visited_neighbors:
                    edge_matrix[s][i].status = (EType.CROSS if vertexs[s].d_time < vertexs[i].d_time else EType.FORWARD)

                clock = clock + 1

    @staticmethod       
    def BFS(start,graph):
        '''广度优先遍历'''
        return
        

In [4]:
if __name__ == '__main__':
    graph = Graph_Matrix()
    graph.insert_vertex('A')
    graph.insert_vertex('B')
    graph.insert_vertex('C')
    graph.insert_vertex('D')
    graph.insert_vertex('E')
    graph.insert_vertex('F')
    graph.insert_vertex('G')
    
    graph.insert_edge(-1,-1,0,1)
    graph.insert_edge(-1,-1,0,2)
    graph.insert_edge(-1,-1,0,5)
    graph.insert_edge(-1,-1,3,0)
    graph.insert_edge(-1,-1,3,4)
    graph.insert_edge(-1,-1,4,5)
    graph.insert_edge(-1,-1,5,6)
    graph.insert_edge(-1,-1,1,2)
    graph.insert_edge(-1,-1,6,2)
    
    def visit(index,vertex):
        print 'index->vertex.data: %s->%s, dtime:%s, ftime:%s'%(index,vertex.data,vertex.d_time,vertex.f_time)
    GraphTraverse.DFS(3,graph,visit)

index->vertex.data: 2->C, dtime:5, ftime:6
index->vertex.data: 6->G, dtime:4, ftime:7
index->vertex.data: 5->F, dtime:3, ftime:8
index->vertex.data: 4->E, dtime:2, ftime:9
index->vertex.data: 1->B, dtime:10, ftime:11
index->vertex.data: 0->A, dtime:2, ftime:12
index->vertex.data: 3->D, dtime:1, ftime:13


#### 三. 拓扑排序  
1. 何为拓扑排序  
 给定一个实际应用, 依赖时间发生顺序形成有向图, 将该有向图在'相容'的条件下, 转换成一个线性序列. 该线性序列称为图的'拓扑排序'  
 '相容'指每一顶点, 都不会通过边, 指向此序列中的前驱节点  
2. 任一不含回路的有向无环图, 都尤其对应的拓扑排序
 有向无环图的拓扑排序并不唯一  
3. 一种递归实现的拓扑排序  
 因为有向无环图都至少含有1个入度为0的点.(否则的话, 图存在回路). 因此, 将该入度为0的点和从其出发的边从图中删去后, 剩下的图形仍然是有向无环图, 依然存在拓扑序列. 因此, 从递归的角度看:  
   1. 递归基 : 只剩一个顶点时, 拓扑序列为该顶点  
   2. 递归条件 : 每次从图中删除入度为0的顶点, 剩下的图队规执行进行拓扑排序
4. 使用深度优先遍历进行拓扑排序 
  1. 上面拓扑排序中, 不断删除入度为0的点进行递归. 现在, 假如我们每次从图中删除出度为0的顶点, 则这个删除的顺序构成拓扑排序的逆序
  1. 深度优先遍历, 在顶点没有UNDISCOVERED状态的邻居时可以访问该顶点, 恰恰能满足了删除出度为0的顶点构成拓扑逆序的要求. 只是从入度为0的顶点开始深度优先遍历, 从而得到拓扑排序的逆序  
    1. python使用list[::-1]反转list

In [5]:
import numpy as np
class TopologicalSort(object):
    @staticmethod
    def t_sort(graphy,visit):
        n = graphy.n
        zero_out = [None for i in range(n)]
        # 查找第一个出度为0的点的index. 翻转edge矩阵在查找, 相当于查找原来edge矩阵哪一个为全None
        start = np.array(graph.E).T.tolist().index(zero_out)  
        GraphTraverse.DFS(start,graph,visit)

In [6]:
if __name__ == '__main__':
    graph = Graph_Matrix()
    graph.insert_vertex('A')
    graph.insert_vertex('B')
    graph.insert_vertex('C')
    graph.insert_vertex('D')
    graph.insert_vertex('E')
    graph.insert_vertex('F')
    
    graph.insert_edge(-1,-1,0,2)
    graph.insert_edge(-1,-1,0,3)
    graph.insert_edge(-1,-1,1,2)
    graph.insert_edge(-1,-1,2,3)
    graph.insert_edge(-1,-1,2,4)
    graph.insert_edge(-1,-1,2,5)
    graph.insert_edge(-1,-1,4,5)
    
    stack = []
    def visit(index,vertex):
        stack.append(vertex.data)
    TopologicalSort.t_sort(graph,visit)
    print stack[::-1]

['B', 'A', 'C', 'E', 'F', 'D']


#### 四. 双联通分量分解
略

#### 五.优先级搜索
优先级搜索是关于图的变成框架.下面的最小支撑树, 最短路径都可以依靠优先级搜索的变成框架实现  
1. 优先级搜索总体思路  
 首先对所有顶点赋予不同的优先级, 然后随着算法的推进不断修改顶点的优先级. 算法的每一次迭代, 都是选取当时有年纪最高的顶点进行操作  
2. 优先级搜索又叫PFS(priority-first search), 或最佳优先搜索BFS(best-first search)  
3. 优先级初始化  
 通常在算法的初始阶段, 都将顶点的优先级统一设置为最大(sys.maxint). 初始化优先级为最低. 此后, 根据需求减小优先级数值, 表示优先级增大  
4. 编程实现  
  1. 初始化起始点优先级, 状态为VISITED, 父节点为-1
  2. 逐步以迭代方式引入顶点和边, 每次引入优先级最高的顶点s, 按照不同策略更新顶点s临近节点的优先级数. 最终构造一颗遍历树

#### 六. 最小支撑树 - Prim算法
1. 最小生成树的概念  
  1. 若联通图G的某一个联通子图T, T能覆盖G的所有顶点, 则称T为G的一颗生成树   
  2. 同一个图可能有多颗支撑树, 但由于其中的顶点总数均为n, 故其边数也均为n-1   
  3. 生成树的成本, 为各边所采用边的权值之和. 所有生成树中, 成本最小的称为最小生成树   
2. 割的概念  
 图$G=(V;E)$中, 顶点集$V$的某个真子集$U$及其在$V$中的补集{V\U}构成图G的一个"割"  
3. 最小生成树的贪心法实现  
  每次迭代之前, 假设已经得到一个最小生成树的子树${ T }_{ k }=\left\{ { V }_{ k },{ E }_{ k } \right\} $.于是, 将${ V }_{ k }$及其补集构成一个'割'. 只要找到${ V }_{ k }$补集到${ V }_{ k }$的最短跨边${ e }_{ k }=\left( { u }_{ k },{ v }_{ k } \right), \left( { u }_{ k }\notin { V }_{ k },{ v }_{ k }\in { V }_{ k } \right) $, 就可将$u_k$并入${ V }_{ k+1 }$, 边$e_k$并入${ E }_{ k+1 }$, 从而得到新的最小生成树${ T }_{ k+1 }=\left\{ { V }_{ k+1 },{ E }_{ k+1 } \right\} $.  
   1. 最短跨边的寻找  
    ${ V }_{ k }$补集中的每个顶点, 到${ V }_{ k }$中个点的最小距离作为该补集顶点的优先级. 而补集中优先级最小的则为需要并入${ V }_{ k+1 }$的顶点

In [14]:
import sys
class Prim(object):
    @staticmethod
    def dist(i,j,edge_matrix):
        '''2个顶点之间的距离'''
        if i==j:
            dist = 0
        else:
            dist = edge_matrix[i][j].weight if edge_matrix[i][j] is not None else sys.maxint
        return dist
    
    @staticmethod
    def init_priority(src,n,vertexs,edge_matrix):
        n = graph.n
        vertexs = graph.V
        edge_matrix = graph.E
        for j in range(n):
            vertexs[j].priority = Prim.dist(src,j,edge_matrix)   
            
    @staticmethod
    def minimum_span_tree(graph,src_idx):
        n = graph.n
        vertexs = graph.V
        flags = [0 if i!=src_idx else 1 for i in range(n)]
        edge_matrix = graph.E
        Prim.init_priority(src_idx,n,vertexs,edge_matrix)
        # 最短路径树
        route = [src_idx] 
        while len(route)<n :
            shortest_disctance_to_route_tree = {}
            for i,flag in enumerate(flags): 
                if flag==0: # 节点尚未并入最短路径树
                    for j in route: # 节点i到当前最小生成树中最近的距离作为节点i的优先级
                        if (vertexs[i].priority>Prim.dist(i,j,edge_matrix)):
                            vertexs[i].priority=Prim.dist(i,j,edge_matrix)
                            vertexs[i].parent = j
                    distance_i = vertexs[i].priority
                    shortest_disctance_to_route_tree[i] = distance_i
                    
            sort_distance = sorted(shortest_disctance_to_route_tree.items(),key=lambda d:d[1])
            findout = sort_distance[0][0]
            route.append(findout)  
            flags[findout] = 1
        return route
            

In [17]:
if __name__ == '__main__':
    graph = Graph_Matrix()
    graph.insert_vertex('A')
    graph.insert_vertex('B')
    graph.insert_vertex('C')
    graph.insert_vertex('D')
    graph.insert_vertex('E')
    graph.insert_vertex('F')
    graph.insert_vertex('G')
    graph.insert_vertex('H')
    
    graph.insert_edge(-1,4,0,1,True)
    graph.insert_edge(-1,12,1,2,True)
    graph.insert_edge(-1,6,0,3,True)
    graph.insert_edge(-1,9,2,3,True)
    graph.insert_edge(-1,1,2,4,True)
    graph.insert_edge(-1,2,2,5,True)
    graph.insert_edge(-1,13,3,4,True)
    graph.insert_edge(-1,5,4,5,True)
    graph.insert_edge(-1,2,3,6,True)
    graph.insert_edge(-1,11,4,6,True) 
    graph.insert_edge(-1,8,4,7,True)
    graph.insert_edge(-1,7,5,7,True)
    graph.insert_edge(-1,14,6,7,True)
    graph.insert_edge(-1,10,2,7,True) 
    graph.insert_edge(-1,7,0,6,True)
    
    a =  Prim.minimum_span_tree(graph,0)
    print '图的最小生成树: ',a

图的最小生成树:  [0, 1, 3, 6, 2, 4, 5, 7]


#### 七. 最短路径
1. Dijkstra最短路径   
  1. 最短路径问题关注给定带权图$G=(V,E)$以及源点s, 对于所有其他顶点v ,s到v的最短路径有哪些边构成, 最短通路有多长  
  2. 设顶点s到v的最短路径为$\rho $,若该路径上经过的任意顶点$u$,其在$\rho$上对应的前缀为$\sigma $, 则$\sigma $也是s到u的最短路径之一  
  有了这个思路, 就可以使用贪心迭代求出源点s到其余所有顶点的最短路径. 
2. 编程思路  
  与最小生成树的思路一样, 迭代之前依然假设以求得某个最短路径子树,将剩余节点中, 离源点最近的顶点并入形成新的最短路径树.  
  距离源点最近的点 : 
    1. 设最短路径树中包含顶点v, 比较distance(u,s)与distance(s,v)+distance(v,u)的大小, 最小的作为顶点u的权值  
    2. 权值最小的顶点, 为需要并入新最短路径树的顶点
  
  如下程序展示了该图形的最短路径
  <img src='img/dijkstra.png' height='15%' width='15%'>

In [7]:
import sys
class Dijkstra(object):
    @staticmethod
    def dist(i,j,edge_matrix):
        '''2个顶点之间的距离'''
        if i==j:
            dist = 0
        else:
            dist = edge_matrix[i][j].weight if edge_matrix[i][j] is not None else sys.maxint
        return dist
    
    @staticmethod
    def init_priority(src,n,vertexs,edge_matrix):
        n = graph.n
        vertexs = graph.V
        edge_matrix = graph.E
        for j in range(n):
            vertexs[j].priority = Dijkstra.dist(src,j,edge_matrix)   
            
    @staticmethod
    def route(graph,src_idx):
        n = graph.n
        vertexs = graph.V
        flags = [0 if i!=src_idx else 1 for i in range(n)]
        edge_matrix = graph.E
        Dijkstra.init_priority(src_idx,n,vertexs,edge_matrix)
        # 最短路径树
        route = [src_idx] 
        while len(route)<n :
            shortest_disctance_to_route_tree = {}
            for i,flag in enumerate(flags): 
                if flag==0: # 节点尚未并入最短路径树
                    for j in route: # 最短路径树
                        if (vertexs[i].priority>vertexs[j].priority+Dijkstra.dist(j,i,edge_matrix)):
                            vertexs[i].priority=vertexs[j].priority+Dijkstra.dist(j,i,edge_matrix)
                            vertexs[i].parent = j
                    distance_i = vertexs[i].priority
                    shortest_disctance_to_route_tree[i] = distance_i
                    
            sort_distance = sorted(shortest_disctance_to_route_tree.items(),key=lambda d:d[1])
            findout = sort_distance[0][0]
            route.append(findout)  
            flags[findout] = 1
        return route
            

In [16]:
if __name__ == '__main__':
    graph = Graph_Matrix()
    graph.insert_vertex('A')
    graph.insert_vertex('B')
    graph.insert_vertex('C')
    graph.insert_vertex('D')
    graph.insert_vertex('E')
    graph.insert_vertex('F')
    graph.insert_vertex('G')
    graph.insert_vertex('H')
    
    graph.insert_edge(-1,4,0,1,True)
    graph.insert_edge(-1,12,1,2,True)
    graph.insert_edge(-1,6,0,3,True)
    graph.insert_edge(-1,9,2,3,True)
    graph.insert_edge(-1,1,2,4,True)
    graph.insert_edge(-1,2,2,5,True)
    graph.insert_edge(-1,13,3,4,True)
    graph.insert_edge(-1,5,4,5,True)
    graph.insert_edge(-1,2,3,6,True)
    graph.insert_edge(-1,11,4,6,True) 
    graph.insert_edge(-1,8,4,7,True)
    graph.insert_edge(-1,7,5,7,True)
    graph.insert_edge(-1,14,6,7,True)
    graph.insert_edge(-1,10,2,7,True) 
    graph.insert_edge(-1,7,0,6,True)
    
    a =  Dijkstra.route(graph,0)
    print '源点0到个点的最短路径:',a

源点0到个点的最短路径: [0, 1, 3, 6, 2, 4, 5, 7]
